# data

In [2]:
import pandas as pd
# Available in the github repo : examples/data/BTC_USD-Hourly.csv
url = "https://raw.githubusercontent.com/ClementPerroud/Gym-Trading-Env/main/examples/data/BTC_USD-Hourly.csv"
df = pd.read_csv(url, parse_dates=["date"], index_col= "date")
df.sort_index(inplace= True)
df.dropna(inplace= True)
df.drop_duplicates(inplace=True)
df.head()

,unix,symbol,open,high,low,close,volume,Volume USD
date,,,,,,,,
2018-05-15 06:00:00,1526364000,BTC/USD,8733.86,8796.68,8707.28,8740.99,4906603.14,559.93
2018-05-15 07:00:00,1526367600,BTC/USD,8740.99,8766.00,8721.11,8739.00,2390398.89,273.58
2018-05-15 08:00:00,1526371200,BTC/USD,8739.00,8750.27,8660.53,8728.49,7986062.84,917.79
2018-05-15 09:00:00,1526374800,BTC/USD,8728.49,8754.40,8701.35,8708.32,1593991.98,182.62
2018-05-15 10:00:00,1526378400,BTC/USD,8708.32,8865.00,8695.11,8795.90,11101273.74,1260.69


# features

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33259 entries, 2018-05-15 06:00:00 to 2022-03-01 00:00:00
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unix        33259 non-null  int64  
 1   symbol      33259 non-null  object 
 2   open        33259 non-null  float64
 3   high        33259 non-null  float64
 4   low         33259 non-null  float64
 5   close       33259 non-null  float64
 6   volume      33259 non-null  float64
 7   Volume USD  33259 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 2.3+ MB


In [4]:
# df is a DataFrame with columns : "open", "high", "low", "close", "Volume USD"

# Create the feature : ( close[t] - close[t-1] )/ close[t-1]
df["feature_close"] = df["close"].pct_change()

# Create the feature : open[t] / close[t]
df["feature_open"] = df["open"]/df["close"]

# Create the feature : high[t] / close[t]
df["feature_high"] = df["high"]/df["close"]

# Create the feature : low[t] / close[t]
df["feature_low"] = df["low"]/df["close"]

 # Create the feature : volume[t] / max(*volume[t-7*24:t+1])
df["feature_volume"] = df["Volume USD"] / df["Volume USD"].rolling(7*24).max()

df.dropna(inplace= True) # Clean again !
# Eatch step, the environment will return 5 inputs  : "feature_close", "feature_open", "feature_high", "feature_low", "feature_volume"

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33092 entries, 2018-05-22 05:00:00 to 2022-03-01 00:00:00
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unix            33092 non-null  int64  
 1   symbol          33092 non-null  object 
 2   open            33092 non-null  float64
 3   high            33092 non-null  float64
 4   low             33092 non-null  float64
 5   close           33092 non-null  float64
 6   volume          33092 non-null  float64
 7   Volume USD      33092 non-null  float64
 8   feature_close   33092 non-null  float64
 9   feature_open    33092 non-null  float64
 10  feature_high    33092 non-null  float64
 11  feature_low     33092 non-null  float64
 12  feature_volume  33092 non-null  float64
dtypes: float64(11), int64(1), object(1)
memory usage: 3.5+ MB


# environment

In [6]:
import gymnasium as gym
import gym_trading_env
env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = df, # Your dataset with your custom features
        positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
    )

# run

In [7]:
# Run an episode until it ends :
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)

Market Return : 423.10%   |   Portfolio Return : -87.95%   |   


# render

In [8]:
# At the end of the episode you want to render
env.save_for_render(dir = "render_logs")

UserWarning: [33mWARN: env.save_for_render to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.save_for_render` for environment variables or `env.get_wrapper_attr('save_for_render')` that will search the reminding wrappers.[0m

In [9]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Oct/2023 17:33:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 17:34:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 17:34:17] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
127.0.0.1 - - [15/Oct/2023 17:34:17] "GET /apple-touch-icon.png HTTP/1.1" 404 -
